The goal of this notebook is to create one big dataset of all the Federal data we could use (for the overall classifier). Text data is prepared... somewhere

In [59]:
import pickle
import numpy as np
import pandas as pd
import nltk
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
import pandas as pd
import html2text as h2t
import requests
from datetime import datetime
import re
import base64
import html2text as h2t
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk import FreqDist
from nltk.tag import pos_tag
from nltk import PorterStemmer
import string
#nltk.download('stopwords')
#nltk.download('punkt')
#nltk.download('averaged_perceptron_tagger')

In [60]:
from pymongo import MongoClient
#CEDC_Quigley
# lpP9pCcDxQ2Y0tv8

client = MongoClient('mongodb+srv://ridgebaseaq-docdb-dqrxk.mongodb.net/test'
                        ,username="CEDC_Quigley"
                        ,password="lpP9pCcDxQ2Y0tv8"
                        ,authSource='admin'
                        ,authMechanism='SCRAM-SHA-1')

database = client['legislative_documents']
collection = database['legislative_texts']#'legiscan_bill_metadata']

legtext_corpus = [[0,0,['dummy','entry']]]

count = 0

legtext_corpus = []
for bl in collection.find().batch_size(30):
    if count%1000 == 0:
        print(count)
    count+=1
    
    base64_bytes = bl['doc'].encode('unicode_escape')
    message_bytes = base64.b64decode(bl['doc'])#base64_bytes)
    message = message_bytes.decode('unicode_escape')

    legtext_corpus.append([bl['bill_id'], bl['doc_id'], message])


0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000


In [61]:
#process the downloaded data
textLinetoStartAfter = 'In the Year of Our Lord'
for ind, document in enumerate(legtext_corpus):
    if ind%1000 == 0:
        print(ind)
    text = document[2]
    rawbillText = h2t.html2text(text)
    billText=''
    i=False
    for l in rawbillText.split('\n'):
        #look through every line, looking for textLinetoStartAfter
        if i:
            billText=billText+str(l)+' '
        if textLinetoStartAfter in l:
            i=True
        
            
    #tokenize the text
    tokens = word_tokenize(billText)
    #create a list of parts of speech tags for the tokens
    tags = pos_tag(tokens)
    # convert to lower case
    tokens = [w.lower() for w in tokens]

    # remove punctuation
    table = str.maketrans('', '', string.punctuation)
    strippedBillText = [w.translate(table) for w in tokens]

    # remove remaining tokens that are not alphabetic
    wordsInBill = [word for word in strippedBillText if word.isalpha()]

    #define the single character 'words' that need to be removed
    singleCharacters = ['a','b','c','d','e','f','g','h','i','j',\
                        'k','l','m','n','o','p','q','r','s','t',\
                        'u','v','w','x','y','z']

    #grab the list of words that were determined to be proper nouns
    #ProperNouns = [word.lower() for word, tag in tags if tag == 'NNP' or tag == 'NNPS' or tag == 'IN']

    #define stop words
    stop_words = set(stopwords.words('english'))
    # filter out stop words, single letter words, and proper nouns
    wordsInBill = [w for w in wordsInBill if not w in stop_words]
    wordsInBill = [w for w in wordsInBill if not w in singleCharacters]
    #wordsInBill = [w for w in wordsInBill if not w in ProperNouns]

    #roll words up to simpler, more categorical forms to make comparison easier
    # stemming of words
    #porter = PorterStemmer()
    #wordsInBill = [porter.stem(word) for word in wordsInBill]

    #create the corpus object
    
    legtext_corpus[ind][2] =  wordsInBill
    #legdoc = [bill_id,doc_id,wordsInBill]
    #legtext_corpus.append(legdoc)

    
    


In [62]:
print(legtext_corpus[0][2])

['act', 'relative', 'definition', 'pushpolling', 'enacted', 'senate', 'house', 'representatives', 'general', 'court', 'convened', 'definitions', 'pushpolling', 'amend', 'rsa', 'xvii', 'read', 'follows', 'conducting', 'calling', 'manner', 'likely', 'construed', 'voter', 'survey', 'poll', 'gather', 'statistical', 'data', 'entities', 'organizations', 'acting', 'independent', 'particular', 'political', 'party', 'candidate', 'interest', 'group', 'part', 'series', 'like', 'telephone', 'calls', 'consist', 'connected', 'calls', 'last', 'less', 'minutes', 'presidential', 'gubernatorial', 'united', 'states', 'senatorial', 'united', 'states', 'congressional', 'elections', 'conducting', 'calling', 'part', 'series', 'like', 'telephone', 'calls', 'consist', 'connected', 'calls', 'last', 'less', 'minutes', 'executive', 'council', 'state', 'senate', 'city', 'town', 'school', 'district', 'village', 'district', 'elections', 'conducting', 'calling', 'part', 'series', 'like', 'telephone', 'calls', 'consis

In [63]:
with open('NHlegtext09_20.corpus','wb') as f:
    pickle.dump(legtext_corpus, f)

In [65]:
with open('NHlegtext09_20.corpus','rb') as f:
    leg_corp = pickle.load(f)
print(leg_corp[0:3])

#we could still stem all these words

[[576685, 1000330, ['act', 'relative', 'definition', 'pushpolling', 'enacted', 'senate', 'house', 'representatives', 'general', 'court', 'convened', 'definitions', 'pushpolling', 'amend', 'rsa', 'xvii', 'read', 'follows', 'conducting', 'calling', 'manner', 'likely', 'construed', 'voter', 'survey', 'poll', 'gather', 'statistical', 'data', 'entities', 'organizations', 'acting', 'independent', 'particular', 'political', 'party', 'candidate', 'interest', 'group', 'part', 'series', 'like', 'telephone', 'calls', 'consist', 'connected', 'calls', 'last', 'less', 'minutes', 'presidential', 'gubernatorial', 'united', 'states', 'senatorial', 'united', 'states', 'congressional', 'elections', 'conducting', 'calling', 'part', 'series', 'like', 'telephone', 'calls', 'consist', 'connected', 'calls', 'last', 'less', 'minutes', 'executive', 'council', 'state', 'senate', 'city', 'town', 'school', 'district', 'village', 'district', 'elections', 'conducting', 'calling', 'part', 'series', 'like', 'telephone

In [56]:
rawbillText = h2t.html2text(legtext_corpus[0][2])
#print('raw text:', rawbillText)
print(len(rawbillText))
i = False
billText = ''
for l in rawbillText.split('\n'):
    #look through every line, looking for textLinetoStartAfter
    if i:
        billText=billText+str(l)+' '
    if textLinetoStartAfter in l:
        i=True
    
print(len(billText))

4572
3627
